In [1]:
import pandas as pd
import os
import re
import numpy as np
from functools import reduce
import copy
#from collection import defaultdict
import statistics
from bs4 import BeautifulSoup
from pathlib import Path
import os.path
import glob
import csv
from optparse import OptionParser

In [2]:
inputDir = "/home/mlk442/FIND/MicroMGx_Pilot/genomics/antismash_outputs"
strain = "DA976Z1625"
path_gcf = "/home/mlk442/FIND/mlk442/gcfs_data.txt"
outdir = "/home/mlk442/FIND/mlk442/BGC_table"

In [3]:
os.chdir(inputDir)
csv_path =outdir +"/" +strain+"_BGC_table.csv"
csv_path

'/home/mlk442/FIND/mlk442/BGC_table/DA976Z1625_BGC_table.csv'

In [4]:
with open (csv_path, "w") as f:
    csc_write = csv.writer(f)
    title ="BCGs in "+strain
    csv_head = ["BGCs in "+strain]
    csc_write.writerow(csv_head)

In [5]:
gbkfiles = []
for file in glob.glob(strain+"/*.gbk"):
    gbkfiles.append(file)
N_gbk = len(gbkfiles) -1
N_gbk      

50

In [6]:
gbkfiles

['DA976Z1625/c00029_NODE_29...region001.gbk',
 'DA976Z1625/c00194_NODE_19...region001.gbk',
 'DA976Z1625/c00071_NODE_71...region001.gbk',
 'DA976Z1625/c00234_NODE_23...region001.gbk',
 'DA976Z1625/c00024_NODE_24...region001.gbk',
 'DA976Z1625/c00305_NODE_30...region001.gbk',
 'DA976Z1625/c00462_NODE_46...region001.gbk',
 'DA976Z1625/c00522_NODE_52...region001.gbk',
 'DA976Z1625/c00483_NODE_48...region001.gbk',
 'DA976Z1625/c00370_NODE_37...region001.gbk',
 'DA976Z1625/c00499_NODE_49...region001.gbk',
 'DA976Z1625/c00129_NODE_12...region001.gbk',
 'DA976Z1625/c00528_NODE_52...region001.gbk',
 'DA976Z1625/c00382_NODE_38...region001.gbk',
 'DA976Z1625/c00241_NODE_24...region001.gbk',
 'DA976Z1625/c00247_NODE_24...region001.gbk',
 'DA976Z1625/c00220_NODE_22...region001.gbk',
 'DA976Z1625/c00020_NODE_20...region001.gbk',
 'DA976Z1625/c00484_NODE_48...region001.gbk',
 'DA976Z1625/c00582_NODE_58...region001.gbk',
 'DA976Z1625/c00016_NODE_16...region001.gbk',
 'DA976Z1625/c00312_NODE_31...regi

In [7]:
#Read html file
path = strain +"/index.html"
htmlfile = open(path, "r", encoding = "utf-8")


In [8]:
#Creat gcfs(key) to gc(value) dictionary, which is specific to the strain
#Some gcs belong to more than one gcf

gcfsData = pd.read_table(path_gcf, sep = "\t")
dict_gcf = dict(zip(gcfsData.gcf.tolist(), gcfsData.bgcs.tolist()))
dict_gc = dict()
for key in dict_gcf.keys():
    gc_list = re.split(" ",dict_gcf[key])    
    filtered_list = list(filter(lambda x: re.match(strain, x) != None, gc_list))
    if len(filtered_list) >0 :
        for gc in filtered_list:
            if gc in dict_gc.keys():
               dict_gc[gc].append(key)
            else:
                dict_gc[gc] = [key]    


In [9]:
dict_gc

{'DA976Z1625_7': ['NRPS_7', 'Others_61', 'PKSI_26', 'RiPPs_129'],
 'DA976Z1625_34': ['NRPS_371'],
 'DA976Z1625_19': ['NRPS_41'],
 'DA976Z1625_17': ['NRPS_513'],
 'DA976Z1625_9': ['NRPS_610'],
 'DA976Z1625_16': ['NRPS_702'],
 'DA976Z1625_49': ['NRPS_754'],
 'DA976Z1625_48': ['NRPS_874'],
 'DA976Z1625_32': ['NRPS_592'],
 'DA976Z1625_36': ['NRPS_370'],
 'DA976Z1625_33': ['NRPS_1282'],
 'DA976Z1625_6': ['NRPS_569'],
 'DA976Z1625_21': ['NRPS_1499'],
 'DA976Z1625_41': ['NRPS_1767'],
 'DA976Z1625_25': ['NRPS_1854'],
 'DA976Z1625_35': ['NRPS_703'],
 'DA976Z1625_44': ['NRPS_1956'],
 'DA976Z1625_20': ['NRPS_2487'],
 'DA976Z1625_12': ['NRPS_3008'],
 'DA976Z1625_15': ['NRPS_3389', 'PKS-NRP_Hybrids_378', 'PKSI_57'],
 'DA976Z1625_40': ['NRPS_3644'],
 'DA976Z1625_46': ['NRPS_7903'],
 'DA976Z1625_13': ['NRPS_8764'],
 'DA976Z1625_18': ['Others_1'],
 'DA976Z1625_26': ['Others_2'],
 'DA976Z1625_22': ['Others_24'],
 'DA976Z1625_3': ['Others_34'],
 'DA976Z1625_27': ['PKSI_43'],
 'DA976Z1625_38': ['PKSI_37'

In [10]:
 #Process html file   
htmlhandle = htmlfile.read()
soup = BeautifulSoup(htmlhandle, "lxml")
count = 0

In [11]:
#region(key) to genbank(value) dictionary
#warning information dictionary with region as key

dict_genbank = dict()
dict_warn = dict()
genbank = soup.find_all("div")
for i in range(len(genbank)):
    if ("Download region GenBank file" in genbank[i].get_text()):
        for link in genbank[i].find_all('a'):
            genbank_info = genbank[i].find_all('a')
            if len(genbank_info) >2:
                name_genbank = genbank_info[1].get("href")
                region_info2 = re.split("#|r|c",genbank_info[2].get("href"))
                name_region2 = "Region "+str(int(region_info2[2]))+"."+region_info2[3]
                dict_genbank[name_region2] = name_genbank 
        if ("Region on contig edge." in genbank[i].get_text()):
            dict_warn[name_region2] = 0

In [12]:
dict_genbank

{'Region 1.1': 'c00001_NODE_1_...region001.gbk',
 'Region 3.1': 'c00003_NODE_3_...region001.gbk',
 'Region 16.1': 'c00016_NODE_16...region001.gbk',
 'Region 20.1': 'c00020_NODE_20...region001.gbk',
 'Region 24.1': 'c00024_NODE_24...region001.gbk',
 'Region 28.1': 'c00028_NODE_28...region001.gbk',
 'Region 29.1': 'c00029_NODE_29...region001.gbk',
 'Region 32.1': 'c00032_NODE_32...region001.gbk',
 'Region 46.1': 'c00046_NODE_46...region001.gbk',
 'Region 62.1': 'c00062_NODE_62...region001.gbk',
 'Region 71.1': 'c00071_NODE_71...region001.gbk',
 'Region 75.1': 'c00075_NODE_75...region001.gbk',
 'Region 83.1': 'c00083_NODE_83...region001.gbk',
 'Region 106.1': 'c00106_NODE_10...region001.gbk',
 'Region 129.1': 'c00129_NODE_12...region001.gbk',
 'Region 141.1': 'c00141_NODE_14...region001.gbk',
 'Region 145.1': 'c00145_NODE_14...region001.gbk',
 'Region 146.1': 'c00146_NODE_14...region001.gbk',
 'Region 160.1': 'c00160_NODE_16...region001.gbk',
 'Region 173.1': 'c00173_NODE_17...region001.g

In [13]:
dict_warn

{'Region 16.1': 0,
 'Region 20.1': 0,
 'Region 24.1': 0,
 'Region 28.1': 0,
 'Region 29.1': 0,
 'Region 32.1': 0,
 'Region 46.1': 0,
 'Region 62.1': 0,
 'Region 71.1': 0,
 'Region 75.1': 0,
 'Region 83.1': 0,
 'Region 106.1': 0,
 'Region 129.1': 0,
 'Region 141.1': 0,
 'Region 145.1': 0,
 'Region 146.1': 0,
 'Region 160.1': 0,
 'Region 173.1': 0,
 'Region 184.1': 0,
 'Region 194.1': 0,
 'Region 220.1': 0,
 'Region 228.1': 0,
 'Region 234.1': 0,
 'Region 241.1': 0,
 'Region 247.1': 0,
 'Region 253.1': 0,
 'Region 269.1': 0,
 'Region 294.1': 0,
 'Region 305.1': 0,
 'Region 312.1': 0,
 'Region 313.1': 0,
 'Region 316.1': 0,
 'Region 325.1': 0,
 'Region 328.1': 0,
 'Region 370.1': 0,
 'Region 382.1': 0,
 'Region 390.1': 0,
 'Region 449.1': 0,
 'Region 462.1': 0,
 'Region 483.1': 0,
 'Region 484.1': 0,
 'Region 499.1': 0,
 'Region 522.1': 0,
 'Region 528.1': 0,
 'Region 552.1': 0,
 'Region 582.1': 0,
 'Region 602.1': 0,
 'Region 647.1': 0}

In [14]:
df = pd.DataFrame(columns = ["Region","Type", "From", "To", "Contig Edge Warning","GenBank", "ID", "GCFs"])            
items = soup.find_all("tr")
faile=0
successed = 0
j=0


In [15]:
for i in range(len(items)):
    # Extract information from first 4 columns by "Region&nbsp" 
    if "Region&nbsp" in items[i].get_text(): 
        j += 1
        info = re.split("\n",items[i].get_text())
        Region = info[2].replace("&nbsp"," ")
        Type = info[5]
        From = info[7]
        To = info[8]
        #Find correaponding waring information in the warn dictionary
        if(Region in dict_warn.keys()):
            edgeWarn = "Region on contig edge"
        else:
            edgeWarn = ""
        #Creat GeneBank and ID information
        forGen = re.split(" |\.", Region)
        ID = strain +"_"+str(j)
        # Check whether Region exists or not, if not, add a row 
        if Region not in df.index.tolist():
            #Transform gdfs list to string
            if (ID in dict_gc.keys()):
                GCFs = ", ".join(dict_gc[ID])
            else:
                GCFs = ""
            GeneBank = strain +"/"+dict_genbank[Region]    
            # Check if this Genbank file is in the antimash folder or not            
            if Path(GeneBank).is_file():
                df.loc[Region] = [Region,Type,From,To, edgeWarn, GeneBank, ID, GCFs]
                successed += 1
            else:
                faile += 0          

In [16]:
if (successed == N_gbk and faile ==0):
    print (strain +": completed successfully!")
else: 
    print (strain + ": "+str(N_gbk) + "gbk file in the folder, "+ str(success)+" gbk file in the table, " 
           + str(failed)+ " GenBank files can't be find in the directory!")

DA976Z1625: completed successfully!


In [17]:
df.style.hide_index()

Region,Type,From,To,Contig Edge Warning,GenBank,ID,GCFs
Region 1.1,lassopeptide,"48,927","74,296",,DA976Z1625/c00001_NODE_1_...region001.gbk,DA976Z1625_1,RiPPs_3418
Region 3.1,T3PKS,"3,486","44,583",,DA976Z1625/c00003_NODE_3_...region001.gbk,DA976Z1625_2,PKSother_40
Region 16.1,siderophore,1,"11,138",Region on contig edge,DA976Z1625/c00016_NODE_16...region001.gbk,DA976Z1625_3,Others_34
Region 20.1,terpene,"14,267","34,667",Region on contig edge,DA976Z1625/c00020_NODE_20...region001.gbk,DA976Z1625_4,Terpene_83
Region 24.1,bacteriocin,"23,402","33,303",Region on contig edge,DA976Z1625/c00024_NODE_24...region001.gbk,DA976Z1625_5,RiPPs_3
Region 28.1,NRPS-like,"8,227","30,507",Region on contig edge,DA976Z1625/c00028_NODE_28...region001.gbk,DA976Z1625_6,NRPS_569
Region 29.1,"NRPS,T1PKS,lanthipeptide",1,"30,401",Region on contig edge,DA976Z1625/c00029_NODE_29...region001.gbk,DA976Z1625_7,"NRPS_7, Others_61, PKSI_26, RiPPs_129"
Region 32.1,T1PKS,1,"29,380",Region on contig edge,DA976Z1625/c00032_NODE_32...region001.gbk,DA976Z1625_8,PKSI_516
Region 46.1,NRPS,1,"26,207",Region on contig edge,DA976Z1625/c00046_NODE_46...region001.gbk,DA976Z1625_9,NRPS_610
Region 62.1,terpene,"2,497","22,911",Region on contig edge,DA976Z1625/c00062_NODE_62...region001.gbk,DA976Z1625_10,Terpene_264


In [18]:
df.to_csv(outdir +"/"+strain+"_BGC_table.csv", mode = "a", index = 0)

In [19]:
outdir +"/"+strain+"_BGC_table.csv"

'/home/mlk442/FIND/mlk442/BGC_table/DA976Z1625_BGC_table.csv'